In [ ]:
from joblib import load
import pandas as pd
import numpy as np
import os
from  data_utilities import *
import cv2  # OpenCV 用于调整图像大小和颜色处理
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import gc  # 引入垃圾回收模块
from tqdm.auto import tqdm  # 自动适配环境 导入tqdm进度条库
from collections import defaultdict

dataset_name = 'ManySig'
dataset_path='../ManySig.pkl/'

compact_dataset = load_compact_pkl_dataset(dataset_path,dataset_name)

print("数据集发射机数量：",len(compact_dataset['tx_list']),"具体为：",compact_dataset['tx_list'])
print("数据集接收机数量：",len(compact_dataset['rx_list']),"具体为：",compact_dataset['rx_list'])
print("数据集采集天数：",len(compact_dataset['capture_date_list']),"具体为：",compact_dataset['capture_date_list'])


tx_list = compact_dataset['tx_list']
rx_list = compact_dataset['rx_list']
equalized = 0
capture_date_list = compact_dataset['capture_date_list']


n_tx = len(tx_list)
n_rx = len(rx_list)
print(n_tx,n_rx)


train_dates = ['2021_03_01', '2021_03_08', '2021_03_15']  # 设定你想用的训练日期
# X_train, y_train, X_test, y_test = preprocess_dataset_cross_IQ_blocks(
#     compact_dataset, tx_list, rx_list, 
#     train_dates=train_dates, 
#     max_sig=None,  # 或者 1000
#     equalized=0,
#     block_size=250
# )
X_train, y_train, X_test, y_test = preprocess_dataset_cross_IQ_blocks_date_interleaved(
    compact_dataset, tx_list, train_dates=train_dates,
    max_sig=None, equalized=equalized, block_size=240, y=80
)
print("X_train shape:", X_train.shape)  # (num_blocks*block_size, block_size, 2)
print("y_train shape:", y_train.shape)
print("X_test  shape:", X_test.shape)  # (num_blocks*block_size, block_size, 2)
print("y_test  shape:", y_test.shape)


In [ ]:
import numpy as np

# === 参数设置 ===
SNR_dB = 10           # 信噪比
fs = 20e6             # 采样率 (Hz)
fc = 2.4e9            # 载波频率 (Hz)
v = 120               # 速度 (km/h)
Add_noise = False     # 是否添加噪声
Add_doppler = False   # 是否添加多普勒频移

# === 多普勒频移计算 ===
def compute_doppler_shift(v, fc):
    c = 3e8  # 光速
    v = v / 3.6 # 转换为m/s
    return (v / c) * fc

fd = compute_doppler_shift(v, fc)
print(f"[INFO] 多普勒频移 fd = {fd:.2f} Hz")

# === 多普勒变换 ===
def add_doppler_shift(signal, fd, fs):
    num_samples = signal.shape[-1]
    t = np.arange(num_samples) / fs
    doppler_phase = np.exp(1j * 2 * np.pi * fd * t)
    return signal * doppler_phase

# === SNR测量函数 ===
def measure_snr(clean_signal, noisy_signal):
    """
    测量实际SNR
    """
    signal_power = np.mean(np.abs(clean_signal) ** 2)
    noise = noisy_signal - clean_signal
    noise_power = np.mean(np.abs(noise) ** 2)
    
    if noise_power == 0:
        return float('inf')
    
    snr_measured = 10 * np.log10(signal_power / noise_power)
    return snr_measured

# === 复数AWGN噪声添加函数 ===
def add_complex_awgn(signal, snr_db):
    """
    为复数信号添加AWGN噪声
    """
    # 计算信号功率
    signal_power = np.mean(np.abs(signal) ** 2)
    
    # 计算噪声功率
    snr_linear = 10 ** (snr_db / 10)
    noise_power = signal_power / snr_linear
    
    # 生成复数噪声（实部和虚部独立，各占一半功率）
    noise_std = np.sqrt(noise_power / 2)
    noise_real = np.random.normal(0, noise_std, signal.shape)
    noise_imag = np.random.normal(0, noise_std, signal.shape)
    noise = noise_real + 1j * noise_imag
    
    return signal + noise, noise

# === 带验证的预处理函数 ===
def preprocess_iq_data(data_real_imag, snr_db=None, fd=None, fs=None, 
                       add_noise=True, add_doppler=True, verify_snr=True):
    """
    预处理IQ数据：可选择性地添加噪声和多普勒频移
    
    参数:
    - data_real_imag: 输入数据，shape (N, T, 2)
    - snr_db: 目标信噪比(dB)，当add_noise=True时必需
    - fd: 多普勒频移(Hz)，当add_doppler=True时必需
    - fs: 采样率(Hz)，当add_doppler=True时必需
    - add_noise: 是否添加噪声 (默认True)
    - add_doppler: 是否添加多普勒频移 (默认True)
    - verify_snr: 是否验证SNR (仅当add_noise=True时有效)
    
    返回:
    - processed_real_imag: 处理后的数据，shape (N, T, 2)
    - snr_info: SNR验证信息（如果verify_snr=True且add_noise=True）
    """
    # 参数检查
    if add_noise and snr_db is None:
        raise ValueError("当add_noise=True时，必须提供snr_db参数")
    
    if add_doppler and (fd is None or fs is None):
        raise ValueError("当add_doppler=True时，必须提供fd和fs参数")
    
    # Step 1: 转为复数 IQ，shape: (N, T, 2) → (N, T)
    data_complex = data_real_imag[..., 0] + 1j * data_real_imag[..., 1]

    processed = []
    snr_measured_list = []
    
    for i, sig in enumerate(data_complex):
        current_sig = sig.copy()
        
        # Step 2: 添加 AWGN 噪声（可选）
        if add_noise:
            noisy_sig, noise = add_complex_awgn(current_sig, snr_db)
            current_sig = noisy_sig
            
            # SNR验证（可选）
            if verify_snr:
                measured_snr = measure_snr(sig, noisy_sig)
                snr_measured_list.append(measured_snr)
                
                # 每10000个样本打印一次进度
                if i % 10000 == 0 and i > 0:
                    avg_snr = np.mean(snr_measured_list[-10000:])
                    print(f"[验证] 样本 {i}, 平均实测SNR: {avg_snr:.2f} dB")
        
        # Step 3: 添加多普勒频移（可选）
        if add_doppler:
            shifted = add_doppler_shift(current_sig, fd, fs)
            current_sig = shifted
            
        processed.append(current_sig)

    processed = np.array(processed)
    
    # Step 4: 转回 [I, Q] 实数格式
    processed_real_imag = np.stack([processed.real, processed.imag], axis=-1)
    
    # SNR验证总结（仅当添加噪声且启用验证时）
    snr_info = None
    if add_noise and verify_snr and snr_measured_list:
        snr_measured_array = np.array(snr_measured_list)
        snr_info = {
            'target_snr': snr_db,
            'measured_mean': np.mean(snr_measured_array),
            'measured_std': np.std(snr_measured_array),
            'measured_min': np.min(snr_measured_array),
            'measured_max': np.max(snr_measured_array),
            'samples_count': len(snr_measured_array)
        }
        
        print(f"\n=== SNR验证结果 ===")
        print(f"目标SNR: {snr_info['target_snr']} dB")
        print(f"实测平均SNR: {snr_info['measured_mean']:.2f} dB")
        print(f"实测标准差: {snr_info['measured_std']:.2f} dB")
        print(f"实测范围: [{snr_info['measured_min']:.2f}, {snr_info['measured_max']:.2f}] dB")
        print(f"验证样本数: {snr_info['samples_count']}")
    
    return processed_real_imag, snr_info

# === 单样本测试函数 ===
def test_single_sample_snr(data_real_imag, snr_db, num_tests=10):
    """
    对单个样本进行多次SNR测试，确保噪声添加的正确性
    """
    print(f"\n=== 单样本SNR测试 (测试{num_tests}次) ===")
    
    # 取第一个样本
    sample_complex = data_real_imag[0, :, 0] + 1j * data_real_imag[0, :, 1]
    
    measured_snrs = []
    for i in range(num_tests):
        noisy_sample, _ = add_complex_awgn(sample_complex, snr_db)
        measured_snr = measure_snr(sample_complex, noisy_sample)
        measured_snrs.append(measured_snr)
        print(f"测试 {i+1}: 目标SNR={snr_db} dB, 实测SNR={measured_snr:.2f} dB")
    
    measured_snrs = np.array(measured_snrs)
    print(f"\n测试统计:")
    print(f"平均值: {np.mean(measured_snrs):.2f} dB")
    print(f"标准差: {np.std(measured_snrs):.2f} dB")
    print(f"误差范围: ±{np.abs(np.mean(measured_snrs) - snr_db):.2f} dB")

# === 使用示例 ===
if __name__ == "__main__":
    # 假设 X_train, X_test 已定义
    
    # 使用全局变量控制处理选项
    X_train_processed, _ = preprocess_iq_data(
        X_train, snr_db=SNR_dB, fd=fd, fs=fs, 
        add_noise=Add_noise, add_doppler=Add_doppler, verify_snr=False
    )
    
    # 单样本测试（只有当Add_noise=True时才有意义）
    if Add_noise:
        test_single_sample_snr(X_train, SNR_dB, num_tests=5)
    else:
        print("\n=== 跳过单样本SNR测试（未启用噪声添加）===")
    
    # 处理测试集（可以根据需要设置不同的选项）
    print(f"\n=== 处理测试集 ===")
    X_test_processed, test_snr_info = preprocess_iq_data(
        X_test, snr_db=SNR_dB, fd=fd, fs=fs, 
        add_noise=Add_noise, add_doppler=Add_doppler, verify_snr=False
    )
    
    # 查看处理前后前10个点
    print(f"\n=== 信号对比 ===")
    print("原始信号 I 分量：", X_train[0, :10, 0])
    print("处理后信号 I 分量：", X_train_processed[0, :10, 0])
    
    # 验证多普勒频移效果（只有当Add_doppler=True时才有意义）
    if Add_doppler:
        print(f"\n=== 多普勒频移验证 ===")
        original_phase = np.angle(X_train[0, :10, 0] + 1j * X_train[0, :10, 1])
        processed_phase = np.angle(X_train_processed[0, :10, 0] + 1j * X_train_processed[0, :10, 1])
        phase_diff = processed_phase - original_phase
        print("相位变化:", phase_diff)
    else:
        print(f"\n=== 跳过多普勒频移验证（未启用多普勒频移）===")

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
from torch.utils.data import DataLoader, TensorDataset, random_split
from datetime import datetime
from tqdm import tqdm
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import itertools
import json
import random

# 假设 SNR_dB 和 fd 已经定义
SNR_dB = globals().get('SNR_dB', 'no')
fd = globals().get('fd', 'no')

# === 网格搜索参数设置 ===
param_grid = {
    'learning_rate': [1e-4, 5e-4, 1e-3],
    'model_dim': [64, 128, 256],
    'num_layers': [2, 4, 6],
    'num_heads': [4, 8],
    'dropout': [0.1, 0.3, 0.5],
    'batch_size': [128, 256, 512]  # 添加batch_size作为参数
}

# === 其他固定参数 ===
raw_input_dim = 2
num_classes = len(np.unique(y_train))
num_epochs = 50  # 训练轮数
patience = 5
val_ratio = 0.2  # 验证集比例
NUM_RANDOM_COMBINATIONS = 50  # 随机选择的组合数量

class GridSearch:
    def __init__(self, param_grid, num_random_combinations=None):
        self.param_grid = param_grid
        self.param_combinations = list(itertools.product(*param_grid.values()))
        self.param_names = list(param_grid.keys())
        self.results = []
        self.num_random_combinations = num_random_combinations
        
        # 如果指定了随机组合数量，则随机选择
        if num_random_combinations is not None:
            if num_random_combinations < len(self.param_combinations):
                self.selected_combinations = random.sample(
                    self.param_combinations, num_random_combinations
                )
            else:
                self.selected_combinations = self.param_combinations
        else:
            self.selected_combinations = self.param_combinations
    
    def generate_param_sets(self):
        """生成参数组合（随机选择或全部）"""
        for combination in self.selected_combinations:
            params = dict(zip(self.param_names, combination))
            yield params
    
    def add_result(self, params, val_accuracy, test_accuracy, training_time, memory_usage):
        """添加结果"""
        self.results.append({
            'params': params,
            'val_accuracy': val_accuracy,
            'test_accuracy': test_accuracy,
            'training_time': training_time,
            'memory_usage': memory_usage
        })
    
    def get_best_params(self, metric='val_accuracy'):
        """获取最佳参数"""
        if not self.results:
            return None
        best_result = max(self.results, key=lambda x: x[metric])
        return best_result

# === 模型定义 ===
class SignalTransformer(nn.Module):
    def __init__(self, raw_input_dim, model_dim, num_heads, num_layers, num_classes, dropout=0.1):
        super(SignalTransformer, self).__init__()
        self.embedding = nn.Linear(raw_input_dim, model_dim)
        encoder_layer = TransformerEncoderLayer(d_model=model_dim, nhead=num_heads, dropout=dropout, batch_first=True)
        self.encoder = TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(model_dim, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = self.encoder(x)
        x = x[:, -1, :]
        x = self.fc(x)
        return x

def get_gpu_memory():
    """获取GPU内存使用情况"""
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        return torch.cuda.max_memory_allocated() / 1024**3  # GB
    return 0

def train_model(params, train_val_dataset, test_dataset, device):
    """训练单个模型"""
    start_time = datetime.now()
    
    # 解包参数
    learning_rate = params['learning_rate']
    model_dim = params['model_dim']
    num_layers = params['num_layers']
    num_heads = params['num_heads']
    dropout = params['dropout']
    batch_size = params['batch_size']
    
    # 分割训练集和验证集
    train_size = int(0.8 * len(train_val_dataset))
    val_size = len(train_val_dataset) - train_size
    train_dataset, val_dataset = random_split(train_val_dataset, [train_size, val_size])
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # 初始化模型
    model = SignalTransformer(raw_input_dim, model_dim, num_heads, num_layers, num_classes, dropout).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)

    # 重置GPU内存统计
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats()
    
    best_val_accuracy = 0
    best_model_state = None
    patience_counter = 0

    # 训练循环
    for epoch in range(num_epochs):
        # 训练
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        scheduler.step()

        # 验证
        model.eval()
        correct_val, total_val = 0, 0
        with torch.no_grad():
            for val_inputs, val_labels in val_loader:
                val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
                val_outputs = model(val_inputs)
                _, predicted = torch.max(val_outputs, 1)
                total_val += val_labels.size(0)
                correct_val += (predicted == val_labels).sum().item()
        
        val_accuracy = 100.0 * correct_val / total_val
        
        # 早停
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model_state = model.state_dict().copy()
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            break

    # 获取内存使用情况
    memory_usage = get_gpu_memory()

    # 加载最佳模型进行测试
    if best_model_state:
        model.load_state_dict(best_model_state)
    
    # 测试集评估
    model.eval()
    correct_test, total_test = 0, 0
    test_preds, test_true = [], []
    
    with torch.no_grad():
        for test_inputs, test_labels in test_loader:
            test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)
            test_outputs = model(test_inputs)
            _, predicted = torch.max(test_outputs, 1)
            total_test += test_labels.size(0)
            correct_test += (predicted == test_labels).sum().item()
            test_preds.extend(predicted.cpu().numpy())
            test_true.extend(test_labels.cpu().numpy())
    
    test_accuracy = 100.0 * correct_test / total_test
    training_time = (datetime.now() - start_time).total_seconds()
    
    return best_val_accuracy, test_accuracy, training_time, memory_usage, test_preds, test_true

def main():
    # === 创建保存目录 ===
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    script_name = "wisig_random_search"
    folder_name = f"{timestamp}_{script_name}_SNR{SNR_dB}dB_fd{fd}_classes_{num_classes}_random{NUM_RANDOM_COMBINATIONS}"
    save_folder = os.path.join(os.getcwd(), "search_results", folder_name)
    os.makedirs(save_folder, exist_ok=True)

    # 保存参数网格
    with open(os.path.join(save_folder, "param_grid.json"), "w") as f:
        json.dump(param_grid, f, indent=2)

    # === 准备数据 ===
    X_test_tensor = torch.tensor(X_test_processed, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

    # 合并训练和验证数据
    X_train_val_tensor = torch.tensor(X_train_processed, dtype=torch.float32)
    y_train_val_tensor = torch.tensor(y_train, dtype=torch.long)
    train_val_dataset = TensorDataset(X_train_val_tensor, y_train_val_tensor)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # === 执行随机搜索 ===
    total_possible_combinations = len(list(itertools.product(*param_grid.values())))
    
    # 设置随机种子以确保可重复性
    random.seed(7)
    
    grid_search = GridSearch(param_grid, num_random_combinations=NUM_RANDOM_COMBINATIONS)
    total_to_run = len(grid_search.selected_combinations)
    
    print(f"开始随机搜索，从 {total_possible_combinations} 种参数组合中随机选择 {total_to_run} 种")
    print(f"参数网格: {param_grid}")
    print(f"随机种子: 777 (可重复)")

    results_file = os.path.join(save_folder, "search_results.txt")
    with open(results_file, "w") as f:
        f.write(f"=== Random Search Results ===\n")
        f.write(f"Timestamp: {timestamp}\n")
        f.write(f"Total possible combinations: {total_possible_combinations}\n")
        f.write(f"Randomly selected: {total_to_run}\n")
        f.write(f"Random seed: 7\n")
        f.write(f"Parameter grid: {json.dumps(param_grid, indent=2)}\n\n")

    # 保存随机选择的参数组合
    selected_params_list = []
    for i, combination in enumerate(grid_search.selected_combinations):
        params = dict(zip(param_grid.keys(), combination))
        selected_params_list.append(params)
    
    with open(os.path.join(save_folder, "selected_parameters.json"), "w") as f:
        json.dump(selected_params_list, f, indent=2)

    # 记录失败的参数组合（主要是内存不足）
    failed_combinations = []

    for i, params in enumerate(grid_search.generate_param_sets()):
        print(f"\n=== 训练组合 {i+1}/{total_to_run} ===")
        print(f"参数: {params}")
        
        try:
            # 检查GPU内存是否足够（粗略估计）
            if torch.cuda.is_available():
                model_dim = params['model_dim']
                batch_size = params['batch_size']
                num_layers = params['num_layers']
                
                # 粗略的内存需求估计（MB）
                estimated_memory = (model_dim * batch_size * num_layers * 10) / 1024  # 简化估计
                if estimated_memory > 8000:  # 如果估计超过8GB，跳过
                    print(f"跳过：估计内存需求 {estimated_memory:.1f}MB 过高")
                    failed_combinations.append((params, "内存需求过高"))
                    continue
            
            val_accuracy, test_accuracy, training_time, memory_usage, test_preds, test_true = train_model(
                params, train_val_dataset, test_dataset, device
            )
            
            grid_search.add_result(params, val_accuracy, test_accuracy, training_time, memory_usage)
            
            print(f"结果: Val Acc = {val_accuracy:.2f}%, Test Acc = {test_accuracy:.2f}%, "
                  f"Time = {training_time:.1f}s, Memory = {memory_usage:.2f}GB")
            
            # 保存中间结果
            with open(results_file, "a") as f:
                f.write(f"Combination {i+1}:\n")
                f.write(f"Params: {params}\n")
                f.write(f"Val Accuracy: {val_accuracy:.2f}%\n")
                f.write(f"Test Accuracy: {test_accuracy:.2f}%\n")
                f.write(f"Training Time: {training_time:.1f}s\n")
                f.write(f"GPU Memory: {memory_usage:.2f}GB\n")
                f.write("-" * 50 + "\n")
                
        except RuntimeError as e:
            if "out of memory" in str(e):
                print(f"内存不足，跳过该组合: {e}")
                failed_combinations.append((params, "内存不足"))
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
            else:
                print(f"训练失败: {e}")
                failed_combinations.append((params, str(e)))
        except Exception as e:
            print(f"训练失败: {e}")
            failed_combinations.append((params, str(e)))

    # === 保存最终结果 ===
    best_result = grid_search.get_best_params()
    
    # 保存详细结果
    results_summary = {
        'best_parameters': best_result['params'] if best_result else None,
        'best_val_accuracy': best_result['val_accuracy'] if best_result else 0,
        'best_test_accuracy': best_result['test_accuracy'] if best_result else 0,
        'all_results': grid_search.results,
        'failed_combinations': failed_combinations,
        'total_possible_combinations': total_possible_combinations,
        'randomly_selected_count': total_to_run,
        'random_seed': 7
    }
    
    with open(os.path.join(save_folder, "detailed_results.json"), "w") as f:
        json.dump(results_summary, f, indent=2, default=str)

    # 保存最佳参数
    if best_result:
        with open(os.path.join(save_folder, "best_parameters.json"), "w") as f:
            json.dump(best_result['params'], f, indent=2)

    # 输出总结
    print(f"\n=== 随机搜索完成 ===")
    print(f"总可能组合数: {total_possible_combinations}")
    print(f"随机选择组合数: {total_to_run}")
    print(f"成功训练: {len(grid_search.results)}/{total_to_run} 个组合")
    print(f"失败: {len(failed_combinations)} 个组合")
    
    if best_result:
        print(f"最佳参数: {best_result['params']}")
        print(f"最佳验证准确率: {best_result['val_accuracy']:.2f}%")
        print(f"对应测试准确率: {best_result['test_accuracy']:.2f}%")
    else:
        print("没有成功的训练结果")

    # 创建结果可视化
    plot_results(grid_search.results, save_folder)

def plot_results(results, save_folder):
    """绘制结果可视化"""
    if not results:
        return
    
    # 准备数据
    val_accuracies = [r['val_accuracy'] for r in results]
    test_accuracies = [r['test_accuracy'] for r in results]
    training_times = [r['training_time'] for r in results]
    memory_usages = [r['memory_usage'] for r in results]
    batch_sizes = [r['params']['batch_size'] for r in results]
    learning_rates = [r['params']['learning_rate'] for r in results]
    model_dims = [r['params']['model_dim'] for r in results]

    plt.figure(figsize=(20, 15))
    
    # 1. 准确率分布
    plt.subplot(3, 3, 1)
    plt.hist(val_accuracies, bins=20, alpha=0.7, label='Validation')
    plt.hist(test_accuracies, bins=20, alpha=0.7, label='Test')
    plt.xlabel('Accuracy (%)')
    plt.ylabel('Frequency')
    plt.title('Accuracy Distribution')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 2. 验证vs测试准确率
    plt.subplot(3, 3, 2)
    plt.scatter(val_accuracies, test_accuracies, alpha=0.6)
    plt.xlabel('Validation Accuracy (%)')
    plt.ylabel('Test Accuracy (%)')
    plt.title('Validation vs Test Accuracy')
    plt.grid(True, alpha=0.3)
    min_acc = min(min(val_accuracies), min(test_accuracies))
    max_acc = max(max(val_accuracies), max(test_accuracies))
    plt.plot([min_acc, max_acc], [min_acc, max_acc], 'r--', alpha=0.5)
    
    # 3. Batch Size vs 准确率
    plt.subplot(3, 3, 3)
    unique_batches = sorted(set(batch_sizes))
    colors = plt.cm.viridis(np.linspace(0, 1, len(unique_batches)))
    
    for i, batch in enumerate(unique_batches):
        mask = [bs == batch for bs in batch_sizes]
        batch_val_acc = [acc for j, acc in enumerate(val_accuracies) if mask[j]]
        batch_test_acc = [acc for j, acc in enumerate(test_accuracies) if mask[j]]
        plt.scatter(batch_val_acc, batch_test_acc, c=[colors[i]]*len(batch_val_acc), 
                   label=f'BS={batch}', alpha=0.6)
    
    plt.xlabel('Validation Accuracy (%)')
    plt.ylabel('Test Accuracy (%)')
    plt.title('Accuracy by Batch Size')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 4. Batch Size vs 训练时间
    plt.subplot(3, 3, 4)
    plt.scatter(batch_sizes, training_times, alpha=0.6)
    plt.xlabel('Batch Size')
    plt.ylabel('Training Time (s)')
    plt.title('Batch Size vs Training Time')
    plt.grid(True, alpha=0.3)
    
    # 5. Batch Size vs 内存使用
    plt.subplot(3, 3, 5)
    plt.scatter(batch_sizes, memory_usages, alpha=0.6)
    plt.xlabel('Batch Size')
    plt.ylabel('GPU Memory (GB)')
    plt.title('Batch Size vs Memory Usage')
    plt.grid(True, alpha=0.3)
    
    # 6. 学习率vs准确率（按batch_size着色）
    plt.subplot(3, 3, 6)
    scatter = plt.scatter(learning_rates, val_accuracies, c=batch_sizes, alpha=0.6, cmap='viridis')
    plt.colorbar(scatter, label='Batch Size')
    plt.xscale('log')
    plt.xlabel('Learning Rate')
    plt.ylabel('Validation Accuracy (%)')
    plt.title('Learning Rate vs Accuracy')
    plt.grid(True, alpha=0.3)
    
    # 7. 模型维度vs准确率
    plt.subplot(3, 3, 7)
    plt.scatter(model_dims, val_accuracies, alpha=0.6)
    plt.xlabel('Model Dimension')
    plt.ylabel('Validation Accuracy (%)')
    plt.title('Model Dimension vs Accuracy')
    plt.grid(True, alpha=0.3)
    
    # 8. 训练时间vs准确率
    plt.subplot(3, 3, 8)
    plt.scatter(training_times, val_accuracies, alpha=0.6)
    plt.xlabel('Training Time (s)')
    plt.ylabel('Validation Accuracy (%)')
    plt.title('Training Time vs Accuracy')
    plt.grid(True, alpha=0.3)
    
    # 9. 内存使用vs准确率
    plt.subplot(3, 3, 9)
    plt.scatter(memory_usages, val_accuracies, alpha=0.6)
    plt.xlabel('GPU Memory (GB)')
    plt.ylabel('Validation Accuracy (%)')
    plt.title('Memory Usage vs Accuracy')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_folder, 'random_search_analysis.png'), dpi=300, bbox_inches='tight')
    plt.close()

if __name__ == "__main__":
    main()